In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split
import random
import numpy as np
from tqdm import tqdm
import argparse
import wandb
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
from model import *
from dataset import *
from utils import *
from sklearn.metrics import precision_score, recall_score, f1_score


In [2]:
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
pos_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_pos/rgb_images/'
neg_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_neg/rgb_images/'

pos_dem_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_pos/dem/'
neg_dem_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_neg/dem/'

pos_gt_mask_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_pos/ground_truth/'
neg_gt_mask_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_neg/ground_truth/'

In [13]:
arg_batch_size = 128
learning_rate = 0.0001
epochs = 1
number_of_workers = 0
# image_size = arg_imagesize
val_percent = 0.1


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

full_dataset = SixImageDataset_DEM_GT(pos_dir, 
                             neg_dir, 
                             pos_dem_dir,
                             neg_dem_dir,
                             pos_gt_mask_dir,
                             neg_gt_mask_dir,
                             transform=transform)

In [14]:
n_val = int(len(full_dataset) * val_percent)
n_train = len(full_dataset) - n_val
train, val = random_split(full_dataset, [n_train, n_val])

train_loader = DataLoader(train, batch_size=arg_batch_size, shuffle=True, num_workers=number_of_workers, pin_memory=True)
val_loader = DataLoader(val, batch_size=arg_batch_size, shuffle=False, num_workers=number_of_workers, pin_memory=True, drop_last=True)
    
    

In [6]:
model = Gully_Classifier(input_size=6*2048, hidden_size=512, output_size=1)

/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
total_labels = []

for batch in tqdm(train_loader):
    images, dem_images, gt_masks, labels = batch

    # features = [resnet_extractor(image) for image in images]
    # stacked_features = torch.stack(features, dim=1)
    # output = mlp_classifier(stacked_features)
    
    total_labels.extend(labels)

100%|███████████████████████████████████████████| 64/64 [10:43<00:00, 10.06s/it]


In [19]:
np.sum(total_labels)

4051.0

In [21]:
len(total_labels) - np.sum(total_labels)

4080.0

In [22]:
val_total_labels = []

for batch in tqdm(val_loader):
    images, dem_images, gt_masks, labels = batch

    # features = [resnet_extractor(image) for image in images]
    # stacked_features = torch.stack(features, dim=1)
    # output = mlp_classifier(stacked_features)
    
    val_total_labels.extend(labels)

100%|█████████████████████████████████████████████| 7/7 [01:04<00:00,  9.20s/it]


In [24]:
len(val_total_labels)

896

In [26]:
np.sum(val_total_labels)

462.0